In [8]:
import pandas as pd
import os
from pyDOE import *
from scipy.io import netcdf as nc

### Download latest version of params file from google drive
* requires 'publishing' the google drive spreadsheet
* file > publish to web
* then it can be set up to continuously publish the spreadsheet to a stable url (with some latency, maybe 1-2 minutes)

In [2]:
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU/pub?output=csv'
cmd = 'curl '+data_url+' > params.csv'
os.system(cmd)

0

### Read in csv data, filtering by the "include" column

In [4]:
data     = pd.read_csv('params.csv')
included = data['include']==1
params   = data.loc[included,['name','min','max','dim2','location']]
params

,name,min,max,dim2,location
1,displar,4.000000e-01,9.500000e-01,NaN,P
2,dleaf,8.100000e-03,2.430000e-01,NaN,P
4,kmax,1.000000e-08,3.000000e-08,4,P
11,baseflow_scalar,5.000000e-04,1.000000e-01,NaN,N
12,maximum_leaf_wetted_fraction,1.000000e-02,5.000000e-01,NaN,N


### Generate parameter sampling
 * option available for latin hypercube (LHC) or one-at-a-time (OAAT)
 * careful, each time you run this you get a new random draw

In [110]:
sampling_protocol = 'OAAT'

if sampling_protocol == 'LHC':
    #define sample size (number of ensemble members)
    nsamp = 10
    nparam = len(params['name'])  #number of parameters

    # Generate the latin hypercube sample
    lhd = lhs(nparam, samples=int(nsamp))

    # scale according to parameter range
    param_array = ((params['max'] - params['min']).values)*lhd + params['min'].values

elif sampling_protocol == 'OAAT':
    #nan is code for keep the default value
    param_array = np.nan*np.ones([2*nparam,nparam])
    mins_index = (np.arange(0,2*nparam,2),np.arange(0,nparam,1))
    maxs_index = (np.arange(1,2*nparam,2),np.arange(0,nparam,1))
    param_array[mins_index]=params['min']
    param_array[maxs_index]=params['max']

    
# store in a pandas dataframe
psets = pd.DataFrame(data=param_array, index=None, columns=params['name'])
~np.isnan(psets['dleaf'][0])

False

## Generate parameter files
* ### this will overwrite parameter files!!
* ### proceed with caution

In [113]:
#create all the new parameter files, copied from basepftfile
basepftfile = "../basecase/clm5_params.c171117.nc"
prefix      = sampling_protocol
for i in range(nsamp):
    targetpftfile = "../paramfiles/"+prefix+str(i+1).zfill(4)+".nc"
    print(targetpftfile)
    os.system('cp '+basepftfile+' '+targetpftfile)
    

#modify the parameter values
for name,loc in zip(params['name'],params['location']):
    if loc=='P':
        print(name)
        for i in range(nsamp):
            if ~np.isnan(psets[name][i]):
                ct= ct+1
                pftfile = "../paramfiles/"+prefix+str(i+1).zfill(4)+".nc"
                f = nc.netcdf_file(pftfile, 'a') # could probably update this to use xarray
                var = f.variables[name]
                var[:] = psets[name][i] 
                # TODO: pft variations
                f.close()

../paramfiles/OAAT0001.nc
../paramfiles/OAAT0002.nc
../paramfiles/OAAT0003.nc
../paramfiles/OAAT0004.nc
../paramfiles/OAAT0005.nc
../paramfiles/OAAT0006.nc
../paramfiles/OAAT0007.nc
../paramfiles/OAAT0008.nc
../paramfiles/OAAT0009.nc
../paramfiles/OAAT0010.nc
displar
dleaf
kmax


### Generate namelist files

First create files and specify the parameter file

In [115]:
for i in range(nsamp):
    nlfile = "../namelist_mods/"+prefix+str(i+1).zfill(4)+".txt" 
    with open(nlfile,"w") as file:
        pftfile = "e00"+str(i+1)+".nc" # placeholder for now; this need to be an exact path for the namelist to know where to pull the params file
        output = "paramfile='%s'\n" % (pftfile)
        print(output)
        file.write(output)

paramfile='e001.nc'

paramfile='e002.nc'

paramfile='e003.nc'

paramfile='e004.nc'

paramfile='e005.nc'

paramfile='e006.nc'

paramfile='e007.nc'

paramfile='e008.nc'

paramfile='e009.nc'

paramfile='e0010.nc'



Then populate them with parameter values\
I couldn't do this in the same loop as creating them because it was overriding content when iterating over parameters, but there may be a better way

In [118]:
for name,loc in zip(params['name'],params['location']):
    if loc=='N':
        print(name)
        for i in range(nsamp):
            print(i+1)
            nlfile = "../namelist_mods/e00"+str(i+1)+".txt"   
            with open(nlfile,"a") as file: # key is using "a" for append option
                output = "%s=%s\n" % (name, psets[name][i]) # Again, we can round these values if we think that's important
                print(output)
                file.write(output) 

baseflow_scalar
1
baseflow_scalar=0.06492708598089841

2
baseflow_scalar=0.010364258751763182

3
baseflow_scalar=0.098233363381138

4
baseflow_scalar=0.04852271929780149

5
baseflow_scalar=0.02994964534013261

6
baseflow_scalar=0.01778612448033447

7
baseflow_scalar=0.051154473279627154

8
baseflow_scalar=0.08086503289277693

9
baseflow_scalar=0.07993450573965355

10
baseflow_scalar=0.03954599970630231

maximum_leaf_wetted_fraction
1
maximum_leaf_wetted_fraction=0.06498677492688287

2
maximum_leaf_wetted_fraction=0.1641523018303713

3
maximum_leaf_wetted_fraction=0.3067633129646689

4
maximum_leaf_wetted_fraction=0.44892798607756335

5
maximum_leaf_wetted_fraction=0.362619440378379

6
maximum_leaf_wetted_fraction=0.29523400055145715

7
maximum_leaf_wetted_fraction=0.22648668378423226

8
maximum_leaf_wetted_fraction=0.14846043324743308

9
maximum_leaf_wetted_fraction=0.039818367753108264

10
maximum_leaf_wetted_fraction=0.4968366997098795

